---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object

In [9]:
def date_sorter():
    import re
    all_dates = []
    for i in range(len(df)):
        # catch 04/20/2009; 04/20/09; 4/20/09; 4/3/09; 4-3-09;
        if re.search(r'\d{1,2}[-/]\d{1,2}[-/](?:\d{4}|\d{2})', df[i]): 
            date = re.search(r'\d{1,2}[-/]\d{1,2}[-/](?:\d{4}|\d{2})', df[i])
            all_dates.append(date.group())
        # catch Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
        elif re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ] ?\d{1,2}[-, ] ?\d{4}', df[i]):
            date = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ] ?\d{1,2}[-, ] ?\d{4}', df[i])
            all_dates.append(date.group())
        # catch 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        elif re.search(r'\d{1,2} ?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[., ] ?\d{4}', df[i]):
            date = re.search(r'\d{1,2} ?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[., ] ?\d{4}', df[i])
            all_dates.append(date.group())
        # catch Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        elif re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}(th|st|nd|rd), \d{4}', df[i]):
            date = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}(th|st|nd|rd), \d{4}', df[i])
            all_dates.append(date.group())
        # catch Feb 2009; Sep 2009; Oct 2010; August 2009; July, 1900;
        elif re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[, ] ?\d{4}', df[i]):
            date = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[, ] ?\d{4}', df[i])
            all_dates.append(date.group())
        # catch 6/2008; 12/2009
        elif re.search(r'\d{1,2}/\d{4}', df[i]):
            date = re.search(r'\d{1,2}/\d{4}', df[i])
            all_dates.append(date.group())
        # catch 2009; 2010
        else:
            date = re.search(r'\d{4}', df[i])
            all_dates.append(date.group())
            
    all_dates[298] = all_dates[298].replace('Janaury', 'January')
    all_dates[313] = all_dates[313].replace('Decemeber', 'December')
#     all_dates[321] = all_dates[321].replace('2June', '2 June')
    
    result = pd.to_datetime(pd.Series(all_dates), errors = 'coerce') # create a series of dates and convert to pandas datetime

    sorted_index = result.sort_values().index   # sorted all dates and get the indexes
    return pd.Series(sorted_index)   # create a new series using the sorted indexes

date_sorter()




0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64